In [3]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from scipy import integrate

import symbolics
import payoffs
import plotting
import selection_functions

In [4]:
%matplotlib inline
plt.style.use("ggplot")

# Basic simulation

In [3]:
integrate.solve_ivp?

In [5]:
# random initial condition
prng = np.random.RandomState(42)
number_of_genotypes = 4
initial_offspring_share, = prng.dirichlet(np.ones(number_of_genotypes), 1)
y0 = initial_offspring_share[:-1]

# define the selection functions
d1, d3 = 1.5, 1
UGA = lambda x_A: selection_functions.kirkpatrick_selection(x_A, d1)
UgA = lambda x_A: selection_functions.kirkpatrick_selection(x_A, d3)

# define the payoffs
payoff_kernel = payoffs.prisoners_dilemma_payoffs(prng)

f = lambda t, y: symbolics.generalized_sexual_selection(y, UGA, UgA, payoff_kernel)
jac = lambda t, y: symbolics.generalized_sexual_selection_jac(y, UGA, UgA, payoff_kernel)

result = integrate.solve_ivp(f, t_span=(0, 10), y0=y0, rtol=1e-9, atol=1e-12, vectorized=True)

In [6]:
result

  message: 'The solver successfully reached the interval end.'
     nfev: 542
     njev: 0
      nlu: 0
      sol: None
   status: 0
  success: True
        t: array([  0.00000000e+00,   7.77009411e-03,   8.54710352e-02,
         1.79037250e-01,   2.73515217e-01,   3.69043337e-01,
         4.65555115e-01,   5.62990654e-01,   6.61293502e-01,
         7.60410518e-01,   8.60291737e-01,   9.60890256e-01,
         1.06216213e+00,   1.16408873e+00,   1.26663193e+00,
         1.36975321e+00,   1.47341691e+00,   1.57759012e+00,
         1.68224257e+00,   1.78734650e+00,   1.89287656e+00,
         1.99880967e+00,   2.10512492e+00,   2.21180341e+00,
         2.31882818e+00,   2.42618402e+00,   2.53385744e+00,
         2.64183645e+00,   2.75011056e+00,   2.85867061e+00,
         2.96750868e+00,   3.07661804e+00,   3.18599303e+00,
         3.29562898e+00,   3.40552219e+00,   3.51566980e+00,
         3.62606978e+00,   3.73672085e+00,   3.84762245e+00,
         3.95877469e+00,   4.07017830e+00,   4.

# Interactive simulation

In [8]:
# sliders used to control the initial condition
x1_slider = widgets.FloatSlider(value=0.25, min=0.0, max=1.0, step=0.01, description=r"$x_1$")
x2_slider = widgets.FloatSlider(value=0.25, min=0.0, max=1.0, step=0.01, description=r"$x_2$")
x3_slider = widgets.FloatSlider(value=0.25, min=0.0, max=1.0, step=0.01, description=r"$x_3$")

# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=10, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=8, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=6, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=4, min=0, max=100, step=0.1, description=r"$S$")

# slider used to control which selection function is being used
U_slider = widgets.Dropdown(options=["kirkpatrick", "seger", "wright-bergstrom"], index=0, description=r"$U_{\gamma(j)A}$")

# slider that controls the parameters of the selection function
d1_slider = widgets.FloatSlider(value=1, min=0.0, max=10, step=0.05, description=r"$d_1$")
d3_slider = widgets.FloatSlider(value=1, min=0.0, max=10, step=0.05, description=r"$d_3$")

# slider used to control the mutation rate
e_slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=1e-3, description=r"$\epsilon$", readout_format=".3f")

# slider that controls max simulation time
max_time_slider = widgets.IntSlider(value=25, min=1, max=500, description=r"$\max t$")

w = widgets.interactive(plotting.plot_generalized_sexual_selection, x1=x1_slider, x2=x2_slider, x3=x3_slider,
                        selection_function=U_slider, d1=d1_slider, d3=d3_slider, 
                        T=T_slider, R=R_slider, P=P_slider, S=S_slider, 
                        max_time=max_time_slider)
display(w)

A Jupyter Widget

In [8]:
# can get access to the solution!
(solution, optimize_result) = w.result